In [15]:
from PIL import Image, ImageDraw, ImageFont
import os

def generate_test_image(text_to_draw, output_filename="test_image.png", font_size=45):
    """
    Membuat gambar teks tunggal dengan komposisi yang sama seperti dataset latih.

    Args:
        text_to_draw (str): Teks yang ingin digambar.
        output_filename (str): Nama file untuk gambar yang dihasilkan.
        font_size (int): Ukuran font.
    """
    try:
        # Tentukan lokasi font. Pastikan 'arial.ttf' tersedia di sistem Anda.
        font = ImageFont.truetype("arial.ttf", font_size)
    except IOError:
        print("Error: Font 'arial.ttf' tidak ditemukan.")
        print("Pastikan font tersebut tersedia di sistem Anda.")
        return

    # Buat kanvas gambar dengan ukuran yang sama: 300x80 piksel, latar belakang putih.
    img = Image.new("RGB", (300, 80), color=(255, 255, 255))
    draw = ImageDraw.Draw(img)

    # Gambarkan teks dengan warna, posisi, dan font yang sama.
    # Posisi (10, 10) dan warna hitam (0, 0, 0) harus konsisten.
    draw.text((10, 10), text_to_draw, font=font, fill=(0, 0, 0))

    # Simpan gambar di direktori yang sama dengan skrip.
    img.save(output_filename)
    print(f"Gambar uji '{output_filename}' telah berhasil dibuat dengan teks '{text_to_draw}'.")
    
if __name__ == "__main__":
    # --- Contoh Penggunaan ---
    # Teks yang ingin Anda uji
    kata_uji = "raja" 
    
    # Panggil fungsi untuk membuat gambar.
    # Anda bisa menyesuaikan ukuran font agar mirip dengan yang acak pada dataset latih (misalnya, 30-64).
    generate_test_image(kata_uji, font_size=45)

Gambar uji 'test_image.png' telah berhasil dibuat dengan teks 'raja'.


In [ ]:
import torch
import os
import torch.nn as nn
from PIL import Image
import albumentations
import numpy as np
import string
import torch.nn.functional as F
# Import model_wordRec class
from model import model_wordRec

# Load konfigurasi dan definisi dataset (sesuai kode yang Anda berikan)
# Anda perlu memastikan file ini ada dan terimport dengan benar
# from config import DEVICE
# from dataset import dataset as dataset_class

# Dummy class untuk konfigurasi jika file config tidak tersedia
class Config:
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    IMAGE_WIDTH = 300
    IMAGE_HEIGHT = 80
    NUM_CLASSES = 27
    HIDDEN_STATE = 128
    CHECKPOINT_PATH = "checkpoint.pth"

config = Config()
device = torch.device(config.DEVICE)
print(f"Using device: {device}")

# --- Helper functions ---

# Mapping indeks ke karakter (sesuai kode train_model)
idx2char = {i: chr(96 + i) for i in range(1, 27)} # 1='a' ... 26='z'
idx2char[0] = "-" # Blank untuk CTC

# Fungsi untuk mendecode prediksi CTC
def decode_prediction(logits):
    """Greedy decode dari output CTC"""
    out = torch.argmax(logits, dim=2)
    out = out.permute(1, 0)
    results = []
    for seq in out:
        prev = -1
        s = ""
        for idx in seq.cpu().numpy():
            if idx != prev and idx != 0:
                s += idx2char.get(idx, "?")
            prev = idx
        results.append(s)
    return results

# --- Main function for single inference ---
def predict_single_image(image_path, model, device, resize=(300, 80)):
    """
    Melakukan inferensi pada satu gambar.

    Args:
        image_path (str): Jalur (path) ke file gambar.
        model (nn.Module): Model PyTorch yang sudah dilatih.
        device (torch.device): Perangkat (CPU/GPU) untuk menjalankan inferensi.
        resize (tuple): Ukuran gambar yang sesuai dengan input model.

    Returns:
        str: Hasil prediksi teks.
    """
    model.eval()
    
    # 1. Preprocessing gambar
    # Buka gambar dan ubah ke format RGB
    image = Image.open(image_path).convert("RGB")
    
    # Resize gambar
    if resize is not None:
        image = image.resize((resize[0], resize[1]), resample=Image.BILINEAR)
        
    image = np.array(image)
    
    # Normalisasi gambar
    aug = albumentations.Compose([albumentations.Normalize(always_apply=True)])
    augmented = aug(image=image)
    image = augmented['image']
    
    # Ubah dimensi gambar: HxWxC -> CxHxW dan ubah ke tensor
    image = np.transpose(image, (2, 0, 1)).astype(np.float32)
    image_tensor = torch.tensor(image, dtype=torch.float).unsqueeze(0).to(device) # Tambah dimensi batch

    # 2. Inferensi
    with torch.no_grad():
        logits = model(image_tensor)
        log_probs = F.log_softmax(logits, dim=2)

    # 3. Decode hasil prediksi
    predictions = decode_prediction(log_probs)
    
    return predictions[0]

# --- Main script ---
if __name__ == "__main__":
    # Inisialisasi model
    net = model_wordRec(
        num_classes=config.NUM_CLASSES, 
        hidden_state=config.HIDDEN_STATE
    ).to(device)

    # Muat checkpoint model yang telah dilatih
    try:
        if config.CHECKPOINT_PATH:
            print(f"Loading model from {config.CHECKPOINT_PATH}...")
            checkpoint = torch.load(config.CHECKPOINT_PATH, map_location=device)
            net.load_state_dict(checkpoint['model_state_dict'])
            print("Model loaded successfully!")
        else:
            print("Warning: No checkpoint path specified. Using un-trained model.")

    except FileNotFoundError:
        print(f"Error: Checkpoint file '{config.CHECKPOINT_PATH}' not found.")
        print("Please train the model first or provide the correct path to the checkpoint file.")
        exit()
    except Exception as e:
        print(f"An error occurred while loading the model: {e}")
        exit()

    # Tentukan path ke gambar yang ingin Anda uji
    # Ganti "path/to/your/image.jpg" dengan path gambar yang sebenarnya
    test_image_path = "test_image.png"
    
    if not os.path.exists(test_image_path):
        print(f"Error: Test image file '{test_image_path}' not found.")
        exit()

    # Jalankan prediksi
    print(f"\nPerforming prediction on: {test_image_path}")
    predicted_text = predict_single_image(test_image_path, net, device)

    # Tampilkan hasilnya
    print(f"Predicted Text: {predicted_text}")

Using device: cuda
Loading model from checkpoint.pth...
Model loaded successfully!

Performing prediction on: test_image.png


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_22536\556487436.py:109: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(config.CHECKPOINT_PATH, map_locati

Predicted Text: raja


Gambar uji 'test_image.png' telah berhasil dibuat dengan teks 'gemini'.
